In [3]:
import numpy as np 
import pandas as pd
import random

In [4]:
train_x = pd.read_csv("./data/processed_x.csv", delimiter=",", header = None) # load from processed images

In [5]:
train_y = pd.read_csv("./data/train_y.csv", delimiter = ",", header = None)

In [145]:
def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [169]:
class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        print sizes[:-1]
        print sizes[1:]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
#             if test_data:
#                 print "Epoch {0}: {1} / {2}".format(
#                     j, self.evaluate(test_data), n_test)
#             else:
#                 print "Epoch {0} complete".format(j)

            for x,y in test_data:
                print self.feedforward(x)
    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        x = x.reshape((x.shape[0],1))
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
            
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

In [170]:
training_x = train_x.head(100)
training_y = train_y.head(100)
test_x = train_x.tail(100)
test_y = train_y.tail(100)
training_y = map(int, training_y[0].tolist())
test_y = map(int, test_y[0].tolist())
# for i in range(len(training_y)):
#     temp = np.zeros(10)
#     temp[training_y[i]] = 1
#     training_y[i] = temp

training_data = zip(training_x.values, training_y)
test_data = zip(test_x.values, test_y)

In [171]:
neural_net = Network([4096, 30, 10])

[4096, 30]
[30, 10]


In [172]:
neural_net.SGD(training_data, 10, 2, 3.0, test_data=test_data)

c:\python27\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


[[9.99999996e-01 9.99999996e-01 6.93374501e-01 6.93374501e-01
  9.99999996e-01 9.99999996e-01 9.99999996e-01 6.93374501e-01
  6.93374501e-01 6.93374501e-01 9.99999996e-01 6.93374501e-01
  6.93374501e-01 9.99999996e-01 6.93374501e-01 6.93374501e-01
  6.93374501e-01 9.99999996e-01 9.99999996e-01 9.99999996e-01
  9.99999996e-01 6.93374501e-01 6.93374501e-01 6.93374501e-01
  6.93374501e-01 9.99999996e-01 9.99999996e-01 9.99999996e-01
  9.99999996e-01 6.93374501e-01]
 [1.89667345e-04 1.89667345e-04 5.34130340e-01 5.34130340e-01
  1.89667345e-04 1.89667345e-04 1.89667345e-04 5.34130340e-01
  5.34130340e-01 5.34130340e-01 1.89667345e-04 5.34130340e-01
  5.34130340e-01 1.89667345e-04 5.34130340e-01 5.34130340e-01
  5.34130340e-01 1.89667345e-04 1.89667345e-04 1.89667345e-04
  1.89667345e-04 5.34130340e-01 5.34130340e-01 5.34130340e-01
  5.34130340e-01 1.89667345e-04 1.89667345e-04 1.89667345e-04
  1.89667345e-04 5.34130340e-01]
 [9.99999868e-01 9.99999868e-01 7.84925457e-01 7.84925457e-01
  9.

[[9.99999996e-01 9.99999996e-01 6.93374501e-01 6.93374501e-01
  9.99999996e-01 9.99999996e-01 9.99999996e-01 6.93374501e-01
  6.93374501e-01 6.93374501e-01 9.99999996e-01 6.93374501e-01
  6.93374501e-01 9.99999996e-01 6.93374501e-01 6.93374501e-01
  6.93374501e-01 9.99999996e-01 9.99999996e-01 6.93374501e-01
  9.99999996e-01 6.93374501e-01 6.93374501e-01 6.93374501e-01
  9.99999996e-01 9.99999996e-01 9.99999996e-01 9.99999996e-01
  9.99999996e-01 6.93374501e-01]
 [1.89667345e-04 1.89667345e-04 5.34130340e-01 5.34130340e-01
  1.89667345e-04 1.89667345e-04 1.89667345e-04 5.34130340e-01
  5.34130340e-01 5.34130340e-01 1.89667345e-04 5.34130340e-01
  5.34130340e-01 1.89667345e-04 5.34130340e-01 5.34130340e-01
  5.34130340e-01 1.89667345e-04 1.89667345e-04 5.34130340e-01
  1.89667345e-04 5.34130340e-01 5.34130340e-01 5.34130340e-01
  1.89667345e-04 1.89667345e-04 1.89667345e-04 1.89667345e-04
  1.89667345e-04 5.34130340e-01]
 [9.99999868e-01 9.99999868e-01 7.84925457e-01 7.84925457e-01
  9.

[[1.         1.         0.69988453 0.69988453 1.         1.
  1.         0.69988453 0.69988453 0.69988453 1.         0.69988453
  0.69988453 1.         0.69988453 0.69988453 0.69988453 1.
  1.         1.         1.         0.69988453 0.69988453 0.69988453
  0.69988453 1.         1.         1.         1.         0.69988453]
 [1.         1.         0.91368041 0.91368041 1.         1.
  1.         0.91368041 0.91368041 0.91368041 1.         0.91368041
  0.91368041 1.         0.91368041 0.91368041 0.91368041 1.
  1.         1.         1.         0.91368041 0.91368041 0.91368041
  0.91368041 1.         1.         1.         1.         0.91368041]
 [0.99999993 0.99999993 0.79264874 0.79264874 0.99999993 0.99999993
  0.99999993 0.79264874 0.79264874 0.79264874 0.99999993 0.79264874
  0.79264874 0.99999993 0.79264874 0.79264874 0.79264874 0.99999993
  0.99999993 0.99999993 0.99999993 0.79264874 0.79264874 0.79264874
  0.79264874 0.99999993 0.99999993 0.99999993 0.99999993 0.79264874]
 [1.     

[[1.         1.         0.69988453 0.69988453 1.         1.
  1.         0.69988453 0.69988453 0.69988453 0.69988453 0.69988453
  0.69988453 1.         0.69988453 0.69988453 0.69988453 1.
  1.         0.69988453 1.         0.69988453 0.69988453 0.69988453
  0.69988453 1.         1.         1.         1.         0.69988453]
 [1.         1.         0.91368041 0.91368041 1.         1.
  1.         0.91368041 0.91368041 0.91368041 0.91368041 0.91368041
  0.91368041 1.         0.91368041 0.91368041 0.91368041 1.
  1.         0.91368041 1.         0.91368041 0.91368041 0.91368041
  0.91368041 1.         1.         1.         1.         0.91368041]
 [0.99999993 0.99999993 0.79264874 0.79264874 0.99999993 0.99999993
  0.99999993 0.79264874 0.79264874 0.79264874 0.79264874 0.79264874
  0.79264874 0.99999993 0.79264874 0.79264874 0.79264874 0.99999993
  0.99999993 0.79264874 0.99999993 0.79264874 0.79264874 0.79264874
  0.79264874 0.99999993 0.99999993 0.99999993 0.99999993 0.79264874]
 [1.     

...

[[1.         1.         0.71694794 0.71694794 1.         0.71694794
  1.         0.99969366 0.71694794 0.71694794 0.71694794 0.71694794
  0.71694794 1.         0.71694794 0.71694794 0.71694794 1.
  1.         1.         1.         0.71694794 0.71694794 0.71694794
  0.71694794 0.71694794 1.         1.         1.         0.71694794]
 [1.         1.         0.91375916 0.91375916 1.         0.91375916
  1.         0.99999312 0.91375916 0.91375916 0.91375916 0.91375916
  0.91375916 1.         0.91375916 0.91375916 0.91375916 1.
  1.         1.         1.         0.91375916 0.91375916 0.91375916
  0.91375916 0.91375916 1.         1.         1.         0.91375916]
 [0.99999998 0.99999998 0.80839918 0.80839918 0.99999998 0.80839918
  0.99999998 0.9999553  0.80839918 0.80839918 0.80839918 0.80839918
  0.80839918 0.99999998 0.80839918 0.80839918 0.80839918 0.99999998
  0.99999998 0.99999998 0.99999998 0.80839918 0.80839918 0.80839918
  0.80839918 0.80839918 0.99999998 0.99999998 0.99999998 0.808

[[1.         1.         0.71858269 0.71858269 1.         1.
  1.         0.71858269 0.71858269 0.71858269 1.         0.71858269
  0.71858269 1.         0.71858269 0.71858269 0.71858269 1.
  1.         1.         1.         0.71858269 0.71858269 0.71858269
  0.71858269 1.         1.         1.         1.         0.71858269]
 [1.         1.         0.91377218 0.91377218 1.         1.
  1.         0.91377218 0.91377218 0.91377218 1.         0.91377218
  0.91377218 1.         0.91377218 0.91377218 0.91377218 1.
  1.         1.         1.         0.91377218 0.91377218 0.91377218
  0.91377218 1.         1.         1.         1.         0.91377218]
 [0.99999999 0.99999999 0.80973209 0.80973209 0.99999999 0.99999999
  0.99999999 0.80973209 0.80973209 0.80973209 0.99999999 0.80973209
  0.80973209 0.99999999 0.80973209 0.80973209 0.80973209 0.99999999
  0.99999999 0.99999999 0.99999999 0.80973209 0.80973209 0.80973209
  0.80973209 0.99999999 0.99999999 0.99999999 0.99999999 0.80973209]
 [1.     

[[1.         1.         0.71858269 0.71858269 1.         1.
  1.         0.71858269 0.71858269 0.71858269 0.71858269 0.71858269
  0.71858269 1.         0.71858269 0.71858269 0.71858269 1.
  1.         0.71858269 1.         0.71858269 0.71858269 0.71858269
  0.71858269 1.         1.         1.         1.         0.71858269]
 [1.         1.         0.91377218 0.91377218 1.         1.
  1.         0.91377218 0.91377218 0.91377218 0.91377218 0.91377218
  0.91377218 1.         0.91377218 0.91377218 0.91377218 1.
  1.         0.91377218 1.         0.91377218 0.91377218 0.91377218
  0.91377218 1.         1.         1.         1.         0.91377218]
 [0.99999999 0.99999999 0.80973209 0.80973209 0.99999999 0.99999999
  0.99999999 0.80973209 0.80973209 0.80973209 0.80973209 0.80973209
  0.80973209 0.99999999 0.80973209 0.80973209 0.80973209 0.99999999
  0.99999999 0.80973209 0.99999999 0.80973209 0.80973209 0.80973209
  0.80973209 0.99999999 0.99999999 0.99999999 0.99999999 0.80973209]
 [1.     

[[1.         1.         0.72006246 0.72006246 1.         1.
  1.         0.72006246 0.72006246 0.72006246 1.         0.72006246
  0.72006246 1.         0.72006246 0.72006246 0.72006246 1.
  1.         1.         1.         0.72006246 0.72006246 0.72006246
  0.72006246 1.         1.         1.         1.         0.72006246]
 [1.         1.         0.91378518 0.91378518 1.         1.
  1.         0.91378518 0.91378518 0.91378518 1.         0.91378518
  0.91378518 1.         0.91378518 0.91378518 0.91378518 1.
  1.         1.         1.         0.91378518 0.91378518 0.91378518
  0.91378518 1.         1.         1.         1.         0.91378518]
 [0.99999999 0.99999999 0.81092218 0.81092218 0.99999999 0.99999999
  0.99999999 0.81092218 0.81092218 0.81092218 0.99999999 0.81092218
  0.81092218 0.99999999 0.81092218 0.81092218 0.81092218 0.99999999
  0.99999999 0.99999999 0.99999999 0.81092218 0.81092218 0.81092218
  0.81092218 0.99999999 0.99999999 0.99999999 0.99999999 0.81092218]
 [1.     

[[1.         1.         0.72006246 0.72006246 1.         1.
  1.         0.72006246 0.72006246 0.72006246 0.72006246 0.72006246
  0.72006246 1.         0.72006246 0.72006246 0.72006246 1.
  1.         0.72006246 1.         0.72006246 0.72006246 0.72006246
  0.72006246 1.         1.         1.         1.         0.72006246]
 [1.         1.         0.91378518 0.91378518 1.         1.
  1.         0.91378518 0.91378518 0.91378518 0.91378518 0.91378518
  0.91378518 1.         0.91378518 0.91378518 0.91378518 1.
  1.         0.91378518 1.         0.91378518 0.91378518 0.91378518
  0.91378518 1.         1.         1.         1.         0.91378518]
 [0.99999999 0.99999999 0.81092218 0.81092218 0.99999999 0.99999999
  0.99999999 0.81092218 0.81092218 0.81092218 0.81092218 0.81092218
  0.81092218 0.99999999 0.81092218 0.81092218 0.81092218 0.99999999
  0.99999999 0.81092218 0.99999999 0.81092218 0.81092218 0.81092218
  0.81092218 0.99999999 0.99999999 0.99999999 0.99999999 0.81092218]
 [1.     